# **Data Exploration**

In [ ]:
# Cell setup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")

# Đọc dữ liệu
df = pd.read_csv('../data/processed/student-clean.csv')

In [ ]:
df.head()

## **Data overview**

**Basic Information**

In [ ]:
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print(f"Overall size: {df.size}")
print("Each row represents the information of one student in a specific course")

**Data Integrity**

In [ ]:
print("Number of duplicate rows:", df.duplicated().sum())
print("Number of entirely empty rows:", df.isnull().all(axis=1).sum())

1. school - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)
2. sex - student's sex (binary: 'F' - female or 'M' - male)
3. age - student's age (numeric: from 15 to 22)
4. address - student's home address type (binary: 'U' - urban or 'R' - rural)
5. famsize - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)
6. Pstatus - parent's cohabitation status (binary: 'T' - living together or 'A' - apart)
7. Medu - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
8. Fedu - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
9. Mjob - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
10. Fjob - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
11. reason - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
12. guardian - student's guardian (nominal: 'mother', 'father' or 'other')
13. traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
14. studytime - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
15. failures - number of past class failures (numeric: n if 1<=n<3, else 4)
16. schoolsup - extra educational support (binary: yes or no)
17. famsup - family educational support (binary: yes or no)
18. paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
19. activities - extra-curricular activities (binary: yes or no)
20. nursery - attended nursery school (binary: yes or no)
21. higher - wants to take higher education (binary: yes or no)
22. internet - Internet access at home (binary: yes or no)
23. romantic - with a romantic relationship (binary: yes or no)
24. famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
25. freetime - free time after school (numeric: from 1 - very low to 5 - very high)
26. goout - going out with friends (numeric: from 1 - very low to 5 - very high)
27. Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
28. Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
29. health - current health status (numeric: from 1 - very bad to 5 - very good)
30. absences - number of school absences (numeric: from 0 to 93)

These grades are related with the course subject, Math or Portuguese:

1. G1 - first period grade (numeric: from 0 to 20)
2. G2 - second period grade (numeric: from 0 to 20)
3. G3 - final grade (numeric: from 0 to 20, output target)
4. Subject: Math or Portuguese

In [ ]:
df.describe()

In [ ]:
y = df["G3"]
print(f"Number of unique values in target variable: {y.nunique()}\nValues: {np.sort(y.unique())}")

In [ ]:
counts = y.value_counts().sort_index()
plt.figure(figsize=(10, 5))
plt.bar(counts.index, counts.values)
plt.xlabel("Values")
plt.ylabel("Frequency")
plt.title("Distribution of Target Variable")
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

## **Numerical Columns Analysis**

In [ ]:
'''
Distribution & Central Tendency:
• What is the distribution shape? (normal, skewed, bimodal, uniform)
• Create visualizations: histograms, box plots, density plots,…
• Calculate: mean, median, standard deviation
'''
# Chọn các cột số quan trọng để phân tích
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Tính toán Central Tendency (Mean, Median, Std) & Shape (Skewness)
stats_summary = df[numeric_cols].agg(['mean', 'median', 'std', 'skew']).T
stats_summary.columns = ['Mean', 'Median', 'Std Dev', 'Skewness']

def determine_shape(skew_val):
    if -0.5 <= skew_val <= 0.5:
        return "Symmetrical (Normal-like)"
    elif skew_val > 0.5:
        return "Right Skewed (Positively)"
    else:
        return "Left Skewed (Negatively)"

stats_summary['Distribution Shape'] = stats_summary['Skewness'].apply(determine_shape)
stats_summary = stats_summary.sort_values(by='Skewness')

print("--- BẢNG THỐNG KÊ MÔ TẢ & HÌNH DÁNG PHÂN PHỐI ---")
display(stats_summary)

In [ ]:
# Lọc ra những cột bị lệch kèm distrubution shape
skewed_cols = stats_summary[stats_summary['Distribution Shape'] != "Symmetrical (Normal-like)"]
print("--- CÁC CỘT BỊ LỆCH VÀ HÌNH DÁNG PHÂN PHỐI ---")
display(skewed_cols)

# Chỉ lấy các biến không có phân phối chuẩn
non_normal_cols = stats_summary[stats_summary['Distribution Shape'] != 'Symmetrical (Normal-like)'].index.tolist()

# Trực quan hóa: Histogram cho các biến không chuẩn
n = len(non_normal_cols)
plots_per_fig = 4
for i in range(0, n, plots_per_fig):
    cols_to_plot = non_normal_cols[i:i+plots_per_fig]
    n_subplots = len(cols_to_plot)
    nrows = 2
    ncols = 2
    fig, axes = plt.subplots(nrows, ncols, figsize=(10, 7))
    axes = axes.flatten()
    for j, col in enumerate(cols_to_plot):
        ax = axes[j]
        sns.histplot(df[col], kde=True, bins=10, color='skyblue', edgecolor='black', ax=ax)
        ax.axvline(df[col].mean(), color='red', linestyle='--', label='Mean')
        ax.axvline(df[col].median(), color='green', linestyle='-', label='Median')
        ax.set_title(f'{col} ({stats_summary.loc[col, "Distribution Shape"]})')
        ax.legend()
    # Ẩn subplot thừa nếu số biến không chia hết cho 4
    for k in range(n_subplots, nrows * ncols):
        fig.delaxes(axes[k])
    plt.tight_layout()
    plt.show()

Dựa vào kết quả trực quan, những cột cần chuyển đổi độ lệch là:
- **Thời gian học (`studytime`)** - Right Skewed: Đa số học ít giờ, ít học sinh học nhiều giờ
- **Vắng mặt (`absences`)** - Right Skewed: Đa số vắng ít, ít học sinh vắng nhiều
- **famrel** - Right Skewed: Đa số học sinh có mối quan hệ gia đình tốt, ít học sinh có mối quan hệ gia đình kém
- **Thời gian du lịch (`traveltime`)** - Right Skewed: Đa số học sinh có thời gian đi lại ngắn, ít học sinh có thời gian đi lại dài
- **Lần rớt môn trước (`failures`)** - Right Skewed: Đa số học sinh không rớt, ít học sinh rớt nhiều lần
- **Dalc và Walc (`Dalc`, `Walc`)** - Right Skewed: Đa số học sinh uống ít, ít học sinh uống nhiều
- **Điểm G3 (`G3`)** - Left Skewed nhẹ: Đa số điểm cao, ít học sinh điểm thấp

In [ ]:
'''
Range & Outliers:
• What are the minimum and maximum values?
• Are min/max values reasonable, or do they indicate errors?
• Identify outliers using box plots, IQR method, or z-scores
• Are outliers genuine extreme values or data entry errors?
'''
# 1. Range Analysis: Min & Max Values
# Tính toán min/max cho các cột số
range_df = df[numeric_cols].agg(['min', 'max']).T

# 2. Outlier Detection using IQR Method
# Hàm phát hiện outlier
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound, outliers[column].unique()

# Phân tích outliers cho từng cột số và lưu vào DataFrame
outlier_results = []
for col in numeric_cols:
    count, lower, upper, unique_outliers = detect_outliers_iqr(df, col)
    bounds = f"{lower:.1f} to {upper:.1f}"
    # Chỉ hiển thị vài giá trị outlier đại diện
    if len(unique_outliers) < 10:
        outlier_str = str(sorted(unique_outliers))
    else:
        outlier_str = str(sorted(unique_outliers)[:5]) + "..."
    outlier_results.append({
        'Column': col,
        'Count': count,
        'Bounds (Min-Max)': bounds,
        'Extreme Values Found': outlier_str
    })
outlier_df = pd.DataFrame(outlier_results)
# Gộp hai bảng range_df (min/max) và outlier_df thành một bảng tổng hợp
summary_df = range_df.copy()
summary_df = summary_df.merge(outlier_df.set_index('Column'), left_index=True, right_index=True, how='left')
# Đổi tên cột cho rõ ràng
summary_df = summary_df.rename(columns={
    'min': 'Min',
    'max': 'Max',
    'Count': 'Outlier Count',
    'Bounds (Min-Max)': 'Outlier Bounds',
    'Extreme Values Found': 'Outlier Values',
    'Reasonable?': 'Reasonable? (Manual Check)'
})
print("--- BẢNG TỔNG HỢP MIN/MAX & OUTLIER ---")
display(summary_df.sort_values(by='Outlier Count', ascending=False))

In [ ]:
# Trực quan hóa boxplot chỉ cho các biến có outlier (Outlier Count > 0)
outlier_cols = summary_df[summary_df['Outlier Count'] > 0].index.tolist()
plots_per_fig = 4
n = len(outlier_cols)
for i in range(0, n, plots_per_fig):
    cols_to_plot = outlier_cols[i:i+plots_per_fig]
    n_subplots = len(cols_to_plot)
    nrows = 2
    ncols = 2
    fig, axes = plt.subplots(nrows, ncols, figsize=(10, 7))
    axes = axes.flatten()
    for j, col in enumerate(cols_to_plot):
        ax = axes[j]
        sns.boxplot(x=df[col], ax=ax, color='skyblue', fliersize=5, boxprops=dict(alpha=0.7))
        ax.set_title(f'Boxplot of {col}')
        ax.set_xlabel(col)
    # Ẩn subplot thừa nếu số biến không chia hết cho 4
    for k in range(n_subplots, nrows * ncols):
        fig.delaxes(axes[k])
    plt.tight_layout()
    plt.show()

**Nhận định**
1. Về điểm số (G3): Có 53 học sinh bị điểm 0.
   ->  Đây có thể không phải lỗi nhập liệu mà là học sinh bỏ thi hoặc rớt môn.
   -> Hành động: Cần xem xét mối quan hệ của nhóm này với số buổi vắng (absences).
   -> Trung bình số buổi vắng của nhóm điểm 0: 0.00
   -> Trung bình số buổi vắng của nhóm điểm > 0: 4.67
2. Về tuổi tác: Có 2 học sinh trên 21 tuổi (Max = 22).
   -> Nhận định: Hợp lý trong bối cảnh trường trung học (lưu ban hoặc đi học muộn), không phải lỗi.
3. Về số buổi vắng (absences): Có 54 học sinh vắng trên 15 buổi (Max = 93).
   -> Nhận định: Có thể do nghỉ ốm dài ngày hoặc các lý do cá nhân khác, không phải lỗi.

Từ phần **Distribution & Central Tendency**, có thể xem xét chuyển đổi độ lệch (skewness) cho các biến: `famrel`, `studytime`, `absences`, `G3`, `traveltime`, `failures`, `Dalc`, `Walc` để cải thiện phân phối dữ liệu trước khi xây dựng mô hình dự đoán.

=> Có thể sử dụng Log Transformation với các biến lệch phải và dùng hàm mũ cho cho các biến lệch trái ở bước tiền xử lý sau này.

## **Categorical Columns Analysis**

In [ ]:
df = pd.read_csv("../data/processed/student-clean.csv")

In [ ]:
# Lọc các cột categorical
cat_cols = df.select_dtypes(include=['object']).columns
print(f"Categorical colums: {cat_cols}")

### Value Distribution

In [ ]:
# Hàm phân tích
def analyze_distrubution_cat_col(colname, df):
    print(f"\nCATEGORICAL COLUMN: {colname.upper()}")

    unique_count = df[colname].nunique()
    print(f"- Unique value: {unique_count}")
    
    print(f"- Column distribution:")
    val_counts = df[colname].value_counts()
    distributions = df[colname].value_counts(normalize= True) * 100
    
    for val, count in val_counts.head(5).items():
        percent = distributions[val]
        print(f"{str(val):<10} | {count:>8} | {percent:>7.2f}%")   

In [ ]:
for col in cat_cols:
    analyze_distrubution_cat_col(col, df)

In [ ]:
# Visualization các cột categorical
n_cols = 3
n_rows = math.ceil(len(cat_cols) / n_cols)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))  # Tạo nhiều biểu đồ
axes = axes.flatten() # Làm phẳng mảng axes

for i, colname in enumerate(cat_cols):
    sns.countplot(data=df, x=colname, ax=axes[i], palette="cividis", hue=colname)
    
    axes[i].set_title(f'Distribution of \"{colname}\"')
    axes[i].set_xlabel('')

    for container in axes[i].containers:
        axes[i].bar_label(container)

# Tắt các ô trống nếu số biểu đồ không lấp đầy lưới
for i in range(len(cat_cols), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

**Nhận xét:**
- Cột khá cân bằng: `sex`, `activities`

- Cột mất cân bằng cao (`Pstatus`, `schoolsup`, `higher`): `Pstatus = T` (88%), `schoolsup = no` (88%), `higher = yes` (90%) $\rightarrow$ Những cột này sẽ ít có khả năng phân loại vì không có sự biến thiên mà chỉ thiên về 1 phía. Gây ra Model bias nếu sử dụng làm `Target`

- Cột mất cân bằng vừa: các cột còn lại 

### Data Quality

In [ ]:
# Kiểm tra tính nhất quán của các cột
for colname in cat_cols:
    unique_vals = sorted(df[colname].unique().astype(str))
    print(f"{colname:>10} : {unique_vals}")

In [ ]:
# Kiểm tra có các giá trị bị thiếu hay không
missing_vals = df[cat_cols].isnull().sum()
percent_missing = missing_vals * 100 / len(missing_vals)
print("Percent of missing value in categorical columns:")
for val, pct in percent_missing.items():
    print(f"{val:>10} : {pct} %")

**Nhận xét:** 
- Không có cột nào có giá trị bị thiếu, giá trị bất thường

- Tuy nhiên, đối với cột `Fjob` ta thấy `health` chỉ chiếm số lượng ít (3.93%) và kích thước mẫu của nhóm khá nhỏ (n = 41) nhưng vẫn đủ ngưỡng tối thiểu để thực hiện các so sánh thống kê

### **Nhận xét chung:** 
- Các cột dữ liệu category: `school`, `sex`, `address`, `famsize`, `Pstatus`, `Mjob`, `Fjob`, `reason`, `guardian`, `schoolsup`, `famsup`, `paid`, `activities`, `nursery`, `higher`, `internet`, `romantic`, `subject`

- Các cột đều bị mất cân bằng dữ liệu (trừ cột `sex`, `activities` khá cân bằng)

- Các cột không có các giá trị bất thường, lỗi hay các giá trị hiếm (trừ `Fjob` nhưng nhóm quyết định giữ giá trị hiếm lại mà không gộp chung)

## **Missing Data Analysis**

In [ ]:
missing = df.isnull().sum()
print(missing)

**Nhận xét:**
- Dữ liệu không có giá trị missing nào
- Labels phân bố không đều, giá trị nằm chủ yếu ở khoảng điểm 8 - 18

## **Relationships & Correlations**

In [ ]:
''' 
Preliminary Patterns:
• Calculate correlation matrix for numerical variables
• Create correlation heatmap
• Identify strongly correlated pairs (positive or negative)
• Are there any surprising relationships?
'''
print("--- PHÂN TÍCH TƯƠNG QUAN (CORRELATION ANALYSIS) ---")

# 1. Tính ma trận tương quan (Correlation Matrix)
numeric_df = df.select_dtypes(include=['number'])
corr_matrix = numeric_df.corr()

# 2. Vẽ Heatmap (Biểu đồ nhiệt)
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool)) # Che một nửa tam giác trên để đỡ rối
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap='Purples', vmin=-1, vmax=1, center=0, linewidths=0.5)
plt.title('Ma trận tương quan giữa các biến số (Correlation Heatmap)', fontsize=16)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

# 3. Xác định các cặp tương quan mạnh (Strongly Correlated Pairs)
print("\n--- CÁC CẶP BIẾN CÓ TƯƠNG QUAN MẠNH (|corr| > 0.5) ---")

# Làm phẳng ma trận và sắp xếp
corr_pairs = corr_matrix.unstack()
sorted_pairs = corr_pairs.sort_values(kind="quicksort", ascending=False)

# Lọc bỏ tương quan với chính nó (=1) và các cặp trùng lặp
strong_pairs = sorted_pairs[sorted_pairs != 1.0]
# Chỉ lấy các cặp có độ lớn tương quan > 0.5
strong_pairs = strong_pairs[abs(strong_pairs) > 0.5]
# Loại bỏ các cặp trùng lặp (ví dụ A-B và B-A) -> Chỉ giữ lại mỗi cặp 1 lần
strong_pairs = strong_pairs[~strong_pairs.index.duplicated(keep='first')]
# Lọc thủ công để tránh trùng lặp do unstack (A,B) vs (B,A)
seen = set()
unique_strong_pairs = {}
for idx, val in strong_pairs.items():
    pair = tuple(sorted(idx))
    if pair not in seen:
        seen.add(pair)
        unique_strong_pairs[pair] = val

# In kết quả
if not unique_strong_pairs:
    print("Không có cặp nào có tương quan mạnh trên 0.5 (ngoại trừ các cột điểm số).")
else:
    for (var1, var2), corr_val in unique_strong_pairs.items():
        print(f"- {var1} vs {var2}: {corr_val:.2f}")


**NHẬN XÉT**

1. Tương quan mạnh nhất: Giữa các cột điểm `G1`, `G2`, `G3` (0.80 - 0.91). -> Kết quả học tập có tính ổn định cao.
2. Tương quan gia đình: Trình độ học vấn của Mẹ (`Medu`) và Cha (`Fedu`) tương quan khá cao (0.64).
3. Tương quan thói quen: Uống rượu ngày thường (`Dalc`) và cuối tuần (`Walc`) đi cùng nhau (0.63).
4. Mối quan hệ bất ngờ (Surprising):
   - `failures` (rớt môn) có tương quan ÂM với G3 (khoảng -0.3 đến -0.4). Rớt càng nhiều, điểm càng thấp -> Hợp lý.
   - `studytime` (thời gian học) có tương quan dương yếu với điểm số, nhưng không mạnh như mong đợi.
   - `absences` (vắng mặt) gần như KHÔNG tương quan với điểm số (gần 0). -> Vắng nhiều chưa chắc đã học kém.

In [ ]:
''' 
Cross-tabulations:
• For important categorical × categorical combinations, create frequency tables
• For numerical × categorical combinations, create grouped summary statistics
'''
print("QUAN HỆ GIỮA CÁC BIẾN PHÂN LOẠI (CAT x CAT)")

# Tạo biến 'Pass' (Đậu/Rớt) để phân tích dễ hơn (G3 >= 10 là Đậu theo thang điểm Bồ Đào Nha)
df['Pass_Status'] = df['G3'].apply(lambda x: 'Pass' if x >= 10 else 'Fail')

# Lấy danh sách các cột categorical (không bao gồm Pass_Status vừa tạo)
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
cat_cols = [col for col in cat_cols if col != 'Pass_Status']

print(f"Các cột phân loại: {cat_cols}")
print(f"Sẽ phân tích {len(cat_cols)} cột với biến mục tiêu Pass_Status")

# Phân tích tỷ lệ Pass/Fail cho TẤT CẢ các cột categorical quan trọng
important_cat_cols = ['sex', 'address', 'Pstatus', 'higher', 'internet', 'romantic']

print("\n=== TỶ LỆ PASS/FAIL THEO CÁC BIẾN PHÂN LOẠI ===")

for col in important_cat_cols:
    if col in df.columns:
        ct_table = pd.crosstab(df[col], df['Pass_Status'], normalize='index') * 100
        
        # Tính chi-square test để xem có significant không
        from scipy.stats import chi2_contingency
        chi2, p_value, _, _ = chi2_contingency(pd.crosstab(df[col], df['Pass_Status']))
        significance = f"{col} Có ý nghĩa thống kê" if p_value < 0.05 else f"{col} Không có ý nghĩa thống kê"
        print(f"   Chi-square p-value: {p_value:.4f} -> {significance}")

# Trực quan hóa các mối quan hệ categorical quan trọng
plt.figure(figsize=(15, 10))

for i, col in enumerate(important_cat_cols[:6]):  # Chỉ vẽ 6 cái đầu
    if col in df.columns:
        plt.subplot(2, 3, i + 1)
        
        # Tính tỷ lệ pass cho mỗi nhóm
        pass_rates = df.groupby(col)['Pass_Status'].apply(lambda x: (x == 'Pass').mean() * 100)
        
        # Vẽ bar chart
        bars = plt.bar(pass_rates.index, pass_rates.values, 
                      color=['lightcoral' if rate < 70 else 'lightblue' for rate in pass_rates.values])
        
        plt.title(f'Tỷ lệ Pass theo {col}')
        plt.ylabel('% Pass')
        plt.xticks(rotation=45)
        
        # Thêm số liệu trên cột
        for bar, rate in zip(bars, pass_rates.values):
            plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1, 
                    f'{rate:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# • For numerical × categorical combinations, create grouped summary statistics
# Chọn các cột phân loại quan trọng để phân tích với các biến số
cat_cols = ['sex', 'address', 'famsize', 'Pstatus', 'higher', 'internet', 'romantic']
# Xem với các biến số quan trọng
numeric_cols = ['G3', 'studytime', 'failures']
print("THỐNG KÊ NHÓM GIỮA BIẾN SỐ VÀ PHÂN LOẠI (NUMERIC x CAT)")
for cat_col in cat_cols:
    print(f"\n--- Phân tích theo nhóm cho cột phân loại: {cat_col} ---")
    grouped_stats = df.groupby(cat_col)[numeric_cols].agg(['min','max','mean', 'median', 'std'])
    # Hiển thị bảng thống kê nhóm cứ 3 numeric một lần để tránh quá dài
    for i in range(0, len(numeric_cols), 3):
        display(grouped_stats[numeric_cols[i:i+3]])

# Trực quan hóa phân phối biến số theo các biến phân loại (Box Plots)
plt.figure(figsize=(15, 5 * len(cat_cols)))
for i, cat_col in enumerate(cat_cols):
    for j, num_col in enumerate(numeric_cols):
        plt.subplot(len(cat_cols), len(numeric_cols), i * len(numeric_cols) + j + 1)
        sns.boxplot(x=cat_col, y=num_col, data=df, palette='Set3', legend=False, hue=cat_col)
        plt.title(f'Phân phối {num_col} theo {cat_col}')
        plt.xlabel(cat_col)
        plt.ylabel(num_col)
        plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## **Initial Observation and Insights**

### **Summary**
**1. Key observation:**
- Sự bất thường của Target (`G3`): điểm số có phân phối gần chuẩn nhưng có 53 trường hợp xuất hiện điểm bằng 0. Kèm theo đó là khi `G3` = 0 thì `absences` = 0, điều này là khá vô lý vì theo hành vi học tập, nếu không nghỉ buổi nào nhưng có `G3` = 0. Bên cạnh đó, ta thấy `G1`, `G2` đa số > 0 nếu `G3` = 0  
$\rightarrow$ Có thể là những học sinh này đã bỏ thi, học lực kém, hoặc do các yếu tố tác động khiến cho `G3` bằng 0

- Phân phối lệch của `absences`: bị lệch phải nhiều (Skewness = 3.74). Trung vị là 2 nhưng xuất hiện các outlier rất lớn $\rightarrow$ Điều này cho thấy việc các học sinh có xu hướng ít nghỉ học, có 1 nhóm nhỏ nghỉ học nhiều.

- Sự đồng nhất về nhân khẩu học: có sự mất cân bằng lớn trong các biến: 90% có cha mẹ sống chung (`PStatus` = T), đa số có nguyện vọng học cao hơn (`higher` = yes) và không có tham gia các lớp học thêm (`schoolsup` = no)

**2. Data quality issues:**
- Một vài cột numerical bị lệch $\rightarrow$ Cần có các cách xử lý để không ảnh hưởng đến chất lượng mô hình 

- Imbalanced class: các cột categorical nhưng đã phân tích ở trên `Pstatus`, `higher`, `Fjob` bị mất cân bằng nghiêm trọng $\rightarrow$ Giảm độ tin cậy khi phân tích hành vi của các nhóm này

**3. Necessary preprocessing steps:**
- Data Integration: Gộp hai bộ dữ liệu student-mat và student-por. Thêm cột định danh Subject ('mat'/'por') để phân biệt ngữ cảnh môn học.

- Data cleaning:
    -  Sử dụng *Log Transformation* cho các cột: `studytime`, `absences`, `Walc` và *Power Transformation* cho các cột: `G3`, `famrel` để cải thiện chất lượng mô hình khi dự đoán

- Feature Engineering:
    - Tạo `Total_alc` dựa vào `Dalc` và `Walc`

- Encoding cho các câu hỏi sử dụng Machine Learning: 
    - Sử dụng **Binary Encoding** cho các biến nhị phân (`school`, `sex`, `address`, `famsize`, ... )
    - Sử dụng **One-Hot Encoding** cho các biến có nhiều nhóm (`Mjob`, `Fjob`, `reason`, `guardian`, ... )

**4. Interesting patterns could lead to research questions:**
- Sự tương quan về quá trình học tập `G1`, `G2`, `G3`: các cặp (`G1`,`G2`), (`G2`, `G3`) có sự tương quan cao $\rightarrow$ Nếu có học sinh đạt điểm thấp ở đầu kì thì rất có nhiều khả năng giữa kì và cuối kì điểm cũng sẽ thấp theo.

### **Red flags**
**1. Serious data quality concerns:**
- Sự tồn tại của `G3 = 0` với `absences = 0` là mâu thuẫn logic nghiêm trọng có ảnh hưởng đến việc phân tích hoặc sử dụng các mô hình Machine Learning cho nên cần phải cân nhắc vấn đề loại bỏ hay không
  
**2. Limitations might affect analysis:**  
- Việc mất cân bằng dữ liệu sẽ dẫn đến việc các kết luận hoặc báo cáo liên quan đến các nhóm thiểu số trong dataset (như `higher = no` hoặc `Fedu = health`) sẽ có độ tin cậy thấp hơn nhóm đa số